In [25]:
import pandas as pd

In [35]:
manual_checking = pd.read_csv("manual_checking_output_3rd.csv")
(manual_checking['download_success'] == 1).sum()

1106

In [36]:
dwn_check = {}
for idx, row in manual_checking.iterrows():
    outlet = row['ID of an image'].split("2")[0]
    if outlet not in dwn_check.keys():
        dwn_check[outlet] = [0, 0, 0, 0]  # downloaded img intact and in news, downloaded img is screenshot, downloaded but has problem, fail to download
    if row['download_success'] == 1:
        if row['image_complete'] == "1" and row['NOT_logo'] == "1" and row['NO_distortion'] == "1" and (row['pic_in_news'] == "1" or row['pic_in_old_version'] == "1"):
            dwn_check[outlet][0] += 1
        elif row['pic_is_screenshot'] == "1":
            dwn_check[outlet][1] += 1
        else:
            dwn_check[outlet][2] += 1
    else:
        dwn_check[outlet][3] += 1
dwn_check        

{'atlantict': [0, 0, 0, 5],
 'atlantic': [14, 0, 2, 4],
 'alternet': [2, 0, 0, 23],
 'salon': [24, 0, 1, 0],
 'federalist': [24, 0, 0, 1],
 'freebeacon': [15, 8, 2, 0],
 'gatewaypundit': [18, 0, 2, 5],
 'nationalreview': [16, 0, 3, 6],
 'newsmax': [22, 1, 2, 0],
 'epochtimes': [17, 2, 1, 5],
 'redstate': [8, 0, 0, 17],
 'rightscoop': [25, 0, 0, 0],
 'townhall': [12, 0, 11, 2],
 'westernjournal': [9, 1, 12, 3],
 'bbc': [14, 5, 1, 0],
 'cns': [16, 0, 0, 9],
 'dailybeast': [18, 1, 3, 3],
 'dailywire': [12, 0, 0, 13],
 'infowars': [5, 0, 5, 15],
 'intercept': [13, 1, 0, 11],
 'latimes': [18, 0, 1, 6],
 'nowthis': [7, 6, 7, 0],
 'politicalinsider': [7, 0, 0, 18],
 'realclearpolitics': [5, 5, 1, 14],
 'slate': [35, 1, 5, 9],
 'watimes': [6, 0, 2, 17],
 'weeklystandard': [0, 0, 0, 10],
 'zerohedge': [6, 0, 4, 15],
 'oann': [0, 0, 0, 25],
 'guardian': [19, 2, 2, 2],
 'huffpost': [8, 1, 15, 1],
 'msnbc': [0, 22, 0, 3],
 'newsweek': [23, 0, 2, 0],
 'newyorker': [12, 0, 13, 0],
 'businessinsider'

In [37]:
#check paywall problem
paywall = ['newyorker', 'latimes', 'wsj', 'nyt', 'economist']
download = {outlet:[] for outlet in paywall}
for idx, row in manual_checking.iterrows():
    for outlet in paywall:
        if row['ID of an image'].startswith(outlet):
            #whether successfully downloaded, whether downloaded image is logo
            dwn = 1 if row['download_success'] == 1 else 0
            islogo = 0 if row['NOT_logo'] == 1 else 0
            download[outlet].append((dwn, islogo))
img_analysis = {outlet:[] for outlet in paywall}
for key in download.keys():
    dwn = [0,0,0]
    for item in download[key]:
        if item[0] == 1:
            if item[1] == 0:
                dwn[0] += 1  # download success and not logo
            else:
                dwn[1] += 1  # logo downloaded
        elif item[0] == 0:
            dwn[2] += 1      # fail to download
    img_analysis[key] = dwn
img_analysis

{'newyorker': [25, 0, 0],
 'latimes': [19, 0, 6],
 'wsj': [25, 0, 0],
 'nyt': [25, 0, 0],
 'economist': [22, 0, 3]}

In [38]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import os
def download_image_checking(url, img_name):
    # headers={"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
    # response = requests.get(url, headers=headers)
    response = requests.get(url)
    if response.status_code != 200:
        print(f"STATUS CODE: {response.status_code}")
        return response.status_code
    else:
        try:
            # Open the image from the bytes in memory
            img = Image.open(BytesIO(response.content))
            image_format = img.format.lower()
            img_name = img_name + f'.{image_format}'
            img.save(os.path.join("images/", img_name))
            print(f"   Downloaded: {url}")
            return 1
        except Exception as e:
            return 0

In [39]:
manual_checking['STATUS CODE'] = 0
manual_checking['re_download_success'] = 0
manual_checking['index'] = range(1500)
for idx, row in manual_checking.iterrows():
    img_link = row['original_image_link']
    news_link = row['original_article_link']
    img_name = row['ID of an image']
    try:
        if (img_link.startswith("http:")):
            img_link = img_link.replace("http:", "https:")
        print(f"now: {idx}")
        status = download_image_checking(img_link, img_name)
        if status != 0 and status != 1:
            manual_checking.iloc[idx, len(manual_checking.columns)-3] = status
            continue
        manual_checking.iloc[idx, len(manual_checking.columns)-3] = 200     # status code 200
        if status == 1:
            manual_checking.iloc[idx, len(manual_checking.columns)-2] = 1   # re download successfully
    except Exception as e:
        pass

now: 0
   Downloaded: https://cdn.theatlantic.com/assets/media/img/mt/2017/11/RTS1J98H/facebook.jpg?1510598023
now: 1
   Downloaded: https://cdn.theatlantic.com/assets/media/img/mt/2017/06/AP_16239764578578/facebook.jpg?1498834939
now: 2
   Downloaded: https://cdn.theatlantic.com/assets/media/img/notes/2017/12/AP_97080102815/facebook.jpg?1514477560
now: 3
   Downloaded: https://cdn.theatlantic.com/assets/media/img/mt/2017/07/RTXD2OU-1/facebook.jpg?1501102070
now: 4
   Downloaded: https://cdn.theatlantic.com/assets/media/img/mt/2017/06/RTXZY9S/facebook.jpg?1498432190
now: 5
   Downloaded: https://cdn.theatlantic.com/assets/media/img/mt/2018/01/GettyImages_520141668toned/facebook.jpg?1516647215
now: 6
   Downloaded: https://cdn.theatlantic.com/assets/media/img/mt/2018/02/male_violence_photo/facebook.jpg?1519664841
now: 7
   Downloaded: https://cdn.theatlantic.com/assets/media/img/mt/2018/05/RTS1Q16Q/facebook.jpg?1526674121
now: 8
   Downloaded: https://cdn.theatlantic.com/assets/media/im

In [45]:
(manual_checking['re_download_success'] == 1).sum()

1099

In [42]:
from pandasql import sqldf
status = sqldf('''
SELECT "ID of an image" as ID, "original_image_link" as "link", download_success, "STATUS CODE", "re_download_success", "index"
FROM manual_checking
''')
pd.set_option('display.max_rows', 1500)
status

,ID,link,download_success,STATUS CODE,re_download_success,index
0,atlantict2017_565544f0-c8a4-11e7-b949-49f2311b...,https://cdn.theatlantic.com/assets/media/img/m...,0,200,1,0
1,atlantict2017_a50a1d60-5da5-11e7-8065-c1194289...,https://cdn.theatlantic.com/assets/media/img/m...,0,200,1,1
2,atlantict2017_739a1850-ebeb-11e7-9d5f-49f2311b...,https://cdn.theatlantic.com/assets/media/img/n...,0,200,1,2
3,atlantict2017_0c3bba00-72d9-11e7-b49b-c1194289...,https://cdn.theatlantic.com/assets/media/img/m...,0,200,1,3
4,atlantict2017_82595280-5a68-11e7-871a-c1194289...,https://cdn.theatlantic.com/assets/media/img/m...,0,200,1,4
5,atlantic2018_07f99d30-ffa7-11e7-8c38-d7f3856009ab,https://cdn.theatlantic.com/assets/media/img/m...,0,200,1,5
6,atlantic2018_99df1860-1b18-11e8-8b25-ef989ddb8d11,https://cdn.theatlantic.com/assets/media/img/m...,0,200,1,6
7,atlantic2018_c33ef900-5ad8-11e8-95bb-757c1a0f4d2c,https://cdn.theatlantic.com/assets/media/img/m...,1,200,1,7
8,atlantic2018_1e413e90-65d2-11e8-96a8-57e1131bb7a6,https://cdn.theatlantic.com/assets/media/img/m...,0,200,1,8
9,atlantic2018_baec2120-aba9-11e8-b9fe-d7f3856009ab,https://cdn.theatlantic.com/assets/media/img/m...,0,200,1,9


In [43]:
# manual_checking['index'] = range(1500)
# from pandasql import sqldf
# re_failures = sqldf('''
# SELECT "ID of an image" as ID, "original_image_link" as "link", download_success, "STATUS CODE", "re_download_success", "index"
# FROM manual_checking
# WHERE re_download_success=0
# ''')
# pd.set_option('display.max_rows', 100)
# re_failures

In [44]:
# # Download success but status code != 200
# abnormal1 = sqldf('''
# SELECT *
# FROM status_code
# WHERE download_success = 1 and "STATUS CODE" != 200
# ''')
# abnormal1
# # Download failed but status code == 200
# abnormal2 = sqldf('''
# SELECT *
# FROM status_code
# WHERE download_success = 0 and "STATUS CODE" == 200
# ''')
# abnormal2

In [64]:
def get_wayback_url(image_url):
    # Access the Wayback Machine API to get the closest archive snapshot
    wayback_api_url = f"https://archive.org/wayback/available?url={image_url}"
    response = requests.get(wayback_api_url)
    data = response.json()

    if 'archived_snapshots' in data and data['archived_snapshots']:
        closest_snapshot = data['archived_snapshots']['closest']
        if closest_snapshot['available']:
            wayback_url = closest_snapshot['url']
            print(f"wayback version found: {wayback_url}")
            return wayback_url
            
    print("wayback version not existing")
    return None
    
def download_image_modified(url, img_name):
    # headers={"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
    # response = requests.get(url, headers=headers)
    response = requests.get(url)
    if response.status_code != 200:
        print(f"STATUS CODE: {response.status_code}")
        # try to get the wayback version
        wayback_url = get_wayback_url(url)
        if wayback_url != None:
            return download_image_modified(wayback_url, img_name)
        else:
            return response.status_code
    else:
        # Convert and save the image as PNG
        img_name = img_name + '.png'
        try:
            # Open the image from the bytes in memory
            img = Image.open(BytesIO(response.content))
            img.save(os.path.join("modified_download_img/", img_name), 'PNG')
            print(f"   Downloaded: {url}")
            return 1
        except Exception as e:
            return 0

In [65]:
modified_manual_checking = manual_checking.copy(deep=False)

for idx, row in modified_manual_checking.iterrows():
    img_link = row['original_image_link']
    news_link = row['original_article_link']
    img_name = row['ID of an image']
    try:
        if (img_link.startswith("http:")):
            img_link = img_link.replace("http:", "https:")
        print(f"now: {idx}")
        status = download_image_modified(img_link, img_name)
        if status != 0 and status != 1:
            modified_manual_checking.iloc[idx, len(modified_manual_checking.columns)-3] = status
            continue
        modified_manual_checking.iloc[idx, len(modified_manual_checking.columns)-3] = 200     # status code 200
        if status == 1:
            modified_manual_checking.iloc[idx, len(modified_manual_checking.columns)-2] = 1   # re download successfully
    except Exception as e:
        pass

now: 0
   Downloaded: https://cdn.theatlantic.com/assets/media/img/mt/2017/11/RTS1J98H/facebook.jpg?1510598023
now: 1
   Downloaded: https://cdn.theatlantic.com/assets/media/img/mt/2017/06/AP_16239764578578/facebook.jpg?1498834939
now: 2
   Downloaded: https://cdn.theatlantic.com/assets/media/img/notes/2017/12/AP_97080102815/facebook.jpg?1514477560
now: 3
   Downloaded: https://cdn.theatlantic.com/assets/media/img/mt/2017/07/RTXD2OU-1/facebook.jpg?1501102070
now: 4
   Downloaded: https://cdn.theatlantic.com/assets/media/img/mt/2017/06/RTXZY9S/facebook.jpg?1498432190
now: 5
   Downloaded: https://cdn.theatlantic.com/assets/media/img/mt/2018/01/GettyImages_520141668toned/facebook.jpg?1516647215
now: 6
   Downloaded: https://cdn.theatlantic.com/assets/media/img/mt/2018/02/male_violence_photo/facebook.jpg?1519664841
now: 7
   Downloaded: https://cdn.theatlantic.com/assets/media/img/mt/2018/05/RTS1Q16Q/facebook.jpg?1526674121
now: 8
   Downloaded: https://cdn.theatlantic.com/assets/media/im

In [66]:
(modified_manual_checking['re_download_success'] == 1).sum()

1196

In [69]:
modified_status = sqldf('''
SELECT "ID of an image" as ID, "original_image_link" as "link", download_success, "STATUS CODE", "re_download_success", "index"
FROM modified_manual_checking
''')
modified_status

,ID,link,download_success,STATUS CODE,re_download_success,index
0,atlantict2017_565544f0-c8a4-11e7-b949-49f2311b...,https://cdn.theatlantic.com/assets/media/img/m...,0,200,1,0
1,atlantict2017_a50a1d60-5da5-11e7-8065-c1194289...,https://cdn.theatlantic.com/assets/media/img/m...,0,200,1,1
2,atlantict2017_739a1850-ebeb-11e7-9d5f-49f2311b...,https://cdn.theatlantic.com/assets/media/img/n...,0,200,1,2
3,atlantict2017_0c3bba00-72d9-11e7-b49b-c1194289...,https://cdn.theatlantic.com/assets/media/img/m...,0,200,1,3
4,atlantict2017_82595280-5a68-11e7-871a-c1194289...,https://cdn.theatlantic.com/assets/media/img/m...,0,200,1,4
5,atlantic2018_07f99d30-ffa7-11e7-8c38-d7f3856009ab,https://cdn.theatlantic.com/assets/media/img/m...,0,200,1,5
6,atlantic2018_99df1860-1b18-11e8-8b25-ef989ddb8d11,https://cdn.theatlantic.com/assets/media/img/m...,0,200,1,6
7,atlantic2018_c33ef900-5ad8-11e8-95bb-757c1a0f4d2c,https://cdn.theatlantic.com/assets/media/img/m...,1,200,1,7
8,atlantic2018_1e413e90-65d2-11e8-96a8-57e1131bb7a6,https://cdn.theatlantic.com/assets/media/img/m...,0,200,1,8
9,atlantic2018_baec2120-aba9-11e8-b9fe-d7f3856009ab,https://cdn.theatlantic.com/assets/media/img/m...,0,200,1,9
